<a href="https://colab.research.google.com/github/ck1972/Hands-On-GeoAI1/blob/main/GeoAI_Lab_5a_XML_Land_Cover_Classification_RF_Classifier1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Explainable Machine Learning in Geospatial Analysis: Land Cover Classification**

## Introduction
Explainable ML methods are essential for interpreting the decision-making processes of complex or "black-box" models. They help identify the most influential input features driving model predictions, thereby enhancing the transparency, interpretability, and trustworthiness of machine learning outcomes.

## Imports and Setup
### Import libraries
Import the necessary libraries (pandas, numpy, scikit-learn, rasterio, etc.).

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from matplotlib.colors import from_levels_and_colors
import seaborn as sns
import joblib
import shap

### Mount Google Drive
Next, mount your Google Drive. You will be prompted to authorize access to your Google Drive. Once mounted, you can read/write files in /content/drive/MyDrive.

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

### Define paths and variables
Define the the paths to access your own directory structure in Google Drive. In this tutorial, we use a CSV training dataset (Bul_TrainingData_2024.csv) containing pixel values and their corresponding classes. We will also define the paths to the training datasets and random forest model.


In [ ]:
# Define path that contains the datasets
Sample_Path = '/content/drive/MyDrive/Bulawayo_Dataset_2025/Bul_TA_S2_Pal_2025.csv'
MODEL_PATH = '/content/drive/MyDrive/Bulawayo_Dataset_2025/best_rf_model.pkl'

### Define target and predictor variables
Next, define and specify the overall structure of the land cover classification task. Bands lists the Sentinel-2 spectral bands (e.g., B2, B3, B4) used as input features (predictors) for the model, while LC indicates the target column named “class.”

In [ ]:
# Define target and predictor variables
features = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B11', 'B12', 'HV']  # Feature columns
label = ['class']

Classes = [0, 1, 2, 3, 4, 5]
N_Classes = 6
Names   = ["Bare area", "Built-up", "Cropland", "Grassland", "Woodland", "Water"]
Palette = [
    '#D3D3D3',  # grey for class 0 (Bare area)
    '#FF0000',  # red for class 1 (Built-up)
    '#FFD700',  # gold for class 2 (Cropland)
    '#ADFF2F',  # greenyellow for class 3 (Grassland)
    '#006400',  # darkgreen for class 4 (Woodland)
    '#0000FF'   # blue for class 5 (Water)
]

##  Load and Prepare Training Data
Next, load and prepare the training data. The training data is in a CSV format with columns for each band (B2, B3, etc.) and a class column (land cover type).

In [ ]:
# Load training data as a DataFrame
df = pd.read_csv(Sample_Path)

# Inspect first few rows
print(df.head())

# Separate features (X) and label (y)
X = df[features]
y = df['class']

# Ensure no missing values
print(f"Missing values in features: {X.isnull().sum().sum()}")
print(f"Missing values in label: {y.isnull().sum()}")

# Split into training and testing subsets
# (you can also do cross-validation if you prefer)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print(f"Training samples: {X_train.shape[0]}")
print(f"Testing samples: {X_test.shape[0]}")

## Perform Explainable Machine Learning (xML)
### Introduction
We will use xML methods such as  SHAP (Shapley Additive exPlanations) model to gain insights in the random forest model.

Let's start by loading and extracting the saved random forest model from the dictionary.

In [ ]:
# Load the saved model
package = joblib.load(MODEL_PATH)
rf_model = package["model"]
features = package["features"]
label = package["label"]

print("Model loaded successfully.")

### Random forest feature importance
Display the gloabl random forest feature importance.

In [ ]:
# Random forest feature importance
# Get feature importances
importances = rf_model.feature_importances_

# Create a DataFrame for plotting
importance_df = pd.DataFrame({
    'Feature': features,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

# Plot feature importances using Seaborn
plt.figure(figsize=(10, 6))
sns.barplot(data=importance_df, x='Importance', y='Feature', palette='viridis')
plt.title('Random Forest Feature Importance')
plt.xlabel('Importance Score')
plt.ylabel('Feature')
plt.grid(True)
plt.tight_layout()
plt.show()

### Random forest feature importance
Plot random forest feature importance per land cover class.

In [ ]:
# Import library
from collections import defaultdict

# Number of features and classes
n_features = len(features)
n_classes = len(Names)

# Initialize a matrix to accumulate weighted importances
class_importance_matrix = np.zeros((n_classes, n_features))

# Loop over all trees in the forest
for estimator in rf_model.estimators_:
    importances = estimator.feature_importances_  # (n_features,)

    # Estimate class probabilities from this tree on training samples
    tree = estimator.tree_
    value_at_root = tree.value[0][0]  # shape: (n_classes,)
    class_prob = value_at_root / value_at_root.sum()  # Normalize

    # Add weighted importances to each class
    for class_idx in range(n_classes):
        class_importance_matrix[class_idx] += class_prob[class_idx] * importances

# Normalize importance within each class
class_importance_matrix = class_importance_matrix / class_importance_matrix.sum(axis=1, keepdims=True)

# Build long-form DataFrame for plotting
importance_data = []
for class_idx, class_name in enumerate(Names):
    for feat_idx, feat in enumerate(features):
        importance_data.append({
            "Class": class_name,
            "Feature": feat,
            "Importance": class_importance_matrix[class_idx, feat_idx]
        })

importance_by_class_df = pd.DataFrame(importance_data)

Display the random forest feature importance per class.

In [ ]:
# Plot the figure
plt.figure(figsize=(12, 6))
sns.barplot(
    data=importance_by_class_df,
    x="Importance",
    y="Feature",
    hue="Class",
    palette=Palette
)
plt.title("Random Forest Feature Importance by Land Cover Class (Soft Aggregation)")
plt.xlabel("Normalized Importance")
plt.ylabel("Feature")
plt.legend(title="Land Cover Class", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()

## SHAP (SHapley Additive exPlanations) method
Next, we will use the SHAP method. The SHAP (SHapley Additive exPlanations) method is based on the shapley values, which is a concept from cooperative game theory proposed by Lloyd Shapley (1953).

Let's create an explainer for the RF model. Explainer prints useful information, especially for resolving potential errors.

In [ ]:
# Sample 1,000 rows from the training data for explainability
X_train_sample = X_train.sample(n=1000, random_state=42)

# Create SHAP explainer using the smaller sample
explainer = shap.TreeExplainer(rf_model)

# Compute SHAP values for the sample only
shap_values = explainer(X_train_sample)

### SHAP summary plot
Finally, display SHAP summary plot to gain insights into the random forest model for only 1000 pixel samples.

In [ ]:
# Import the ListedColormap class to create custom colormaps
from matplotlib.colors import ListedColormap

# Define a custom color palette corresponding to different land cover classes
Palette = [
    '#FFD700',  # gold - e.g., Cropland
    '#ADFF2F',  # greenyellow - e.g., Grassland
    '#FF0000',  # red - e.g., Built-up
    '#D3D3D3',  # grey - e.g., Bare area
    '#006400',  # darkgreen - e.g., Woodland
    '#0000FF'   # blue - e.g., Water
]

# Convert the list of hex colors into a matplotlib colormap
my_cmap = ListedColormap(Palette)

# Generate a SHAP summary plot to interpret feature importance
shap.summary_plot(
    shap_values,        # SHAP values for the model predictions
    X_train_sample,     # Subsample of training features
    feature_names=features,   # Names of the input features
    class_names=Names,        # Descriptive labels for each class
    color=my_cmap             # Use the custom colormap for class coloring
)

### SHAP Beeswarm plots by land cover class
Finally, generate individual SHAP beeswarm plots for each land cover class. This enables visualization of how each input feature such as spectral bands and backscatter contributes to the model's prediction for a specific class. We begin by looping through all defined land cover classes (Names) and, for each class, extracts the corresponding SHAP values using the third dimension ([..., class_index]) of the shap_values array. A new shap.Explanation object is then constructed for each class to retain essential metadata like base values and feature names. The shap.summary_plot() function is called with plot_type="dot" to produce a beeswarm plot for the current class. In these plots, each point represents a feature’s SHAP value for a single sample, illustrating both the relative importance of the feature (based on the spread) and the direction of its effect (positive or negative) on the model’s output. This approach provides a clear and interpretable breakdown of the most influential features driving the classification of each land cover type.

In [ ]:
# Loop through each class index and plot a class-specific beeswarm
for class_index, class_name in enumerate(Names):
    print(f"Generating SHAP beeswarm plot for class: {class_name}")

    # Extract SHAP values for the given class
    shap_class_values = shap.Explanation(
        values=shap_values.values[..., class_index],
        base_values=shap_values.base_values[..., class_index],
        data=shap_values.data,
        feature_names=shap_values.feature_names
    )

    # Plot class-specific beeswarm
    shap.summary_plot(
        shap_class_values,
        X_train_sample,
        plot_type="dot",
        show=True
    )